In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

iris = load_iris()
X = iris.data
Y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
Y = encoder.fit_transform(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
input_size = X_train.shape[1]
hidden_size = 10
output_size = Y_train.shape[1]

np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def forward(X):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return A1, A2

def backprop(X, Y, A1, A2, learning_rate=0.1):
    global W1, b1, W2, b2
    m = X.shape[0]
    
    dZ2 = A2 - Y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

In [ ]:
num_epochs = 10000
for epoch in range(num_epochs):
    A1, A2 = forward(X_train)
    backprop(X_train, Y_train, A1, A2)
    if epoch % 1000 == 0:
        loss = np.mean((Y_train - A2) ** 2)
        print(f"Epoch {epoch}, Loss: {loss}")

Epoch 0, Loss: 0.249872677865904
Epoch 1000, Loss: 0.04172626462073474
Epoch 2000, Loss: 0.014486031186898868
Epoch 3000, Loss: 0.012498276650596425
Epoch 4000, Loss: 0.011898005334734759
Epoch 5000, Loss: 0.011619022029999872
Epoch 6000, Loss: 0.011457480563977055
Epoch 7000, Loss: 0.011349963074894437
Epoch 8000, Loss: 0.011270930887059421
Epoch 9000, Loss: 0.011208269404599881


In [ ]:
_, Y_pred = forward(X_test)
Y_pred_labels = np.argmax(Y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

accuracy = np.mean(Y_pred_labels == Y_test_labels)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 1.0


In [8]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    X_new = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    
    _, Y_pred = forward(X_new)
    
    predicted_class = np.argmax(Y_pred, axis=1)[0]
    
    iris_species = {
        0: 'Iris-setosa',
        1: 'Iris-versicolor',
        2: 'Iris-virginica'
    }
    
    probabilities = Y_pred[0]
    
    print(f"Input Features: [Sepal Length: {sepal_length}, Sepal Width: {sepal_width}, "
          f"Petal Length: {petal_length}, Petal Width: {petal_width}]")
    print(f"Predicted Species: {iris_species[predicted_class]}")
    print(f"Confidence scores: Setosa: {probabilities[0]:.4f}, "
          f"Versicolor: {probabilities[1]:.4f}, Virginica: {probabilities[2]:.4f}")
    
    return iris_species[predicted_class]

predict_iris(5.1, 3.5, 1.4, 0.2)

Input Features: [Sepal Length: 5.1, Sepal Width: 3.5, Petal Length: 1.4, Petal Width: 0.2]
Predicted Species: Iris-setosa
Confidence scores: Setosa: 0.9987, Versicolor: 0.0024, Virginica: 0.0000


'Iris-setosa'